In [1]:
#pré-requis : pip install les package : newspaper3k, nltk, pandas

import datetime
import newspaper
import pandas as pd
import numpy as np
from newspaper import Article, Source, Config, version, news_pool
from tqdm import tqdm_notebook
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sources = newspaper.popular_urls()

sources

In [2]:
pd.set_option("display.max_colwidth",100)

In [ ]:
def kw_scraping(keyword,n=5,source_list=[]):
    d=[]
    sources = source_list
   
    if not sources:
        
        sources = newspaper.popular_urls()
    
    
    for url in sources :
        
        journal = newspaper.build(url, memoize_articles=False)
        q = len(journal.articles)
        
        if q != 0:
                        
            m = min(n,q)
            
            for i in range(m):
                                
                article = journal.articles[i]
                
                try:
                    
                    article.download()
                    article.parse()
                    article.nlp()
                    keywords = article.keywords
                    title = article.title
                    
                    if keyword in keywords :
                        d += [[url, article.title, article.text]]
                        
                        #print((i, m, url, "found"))      #test
                        print(article.publish_date)      #test
                        
                        
                    else :
                    
                        #print((i, m, url, "not found"))  #test
                        print(article.publish_date)
                    
                except:
                    print((i, m, url, "error"))          #test
                    continue
                
    df = pd.DataFrame(d, columns=['Source', 'Title', 'Article'])
    df.to_csv("scraping_by_keyword.csv", index = False)
    display(df)

In [ ]:
sourcetest0 = ['http://www.huffingtonpost.com',
 'http://cnn.com',
 'http://www.time.com','http://www.foxnews.com']

sourcetest2 =['http://www.huffingtonpost.com',
 'http://cnn.com',
 'http://www.time.com',
 'http://www.ted.com',
 'http://pandodaily.com',
 'http://www.cnbc.com',
 'http://www.pcmag.com',
 'http://www.foxnews.com',
 'http://www.bbc.co.uk']  #on sait que mlb.com marche pas lors du article.parse, alors que cnn.com oui

In [ ]:
kw_scraping("coronavirus", n=10, source_list = sourcetest2)

In [ ]:
def kw_scraping2(keyword_list, n=1, source_list=[], min_date=[1,1,1], max_date=[9999,12,31], art_per_source= float("inf")):
    
    d=[]
    sources = source_list
   
    if not sources:
            sources = newspaper.popular_urls()
    
    date1 = datetime.datetime(min_date[0],min_date[1],min_date[2])
    date2 = datetime.datetime(max_date[0],max_date[1],max_date[2])
    
    nb_sources = len(sources)
    
    t = 0
    
    stop = False
    
    while not stop :
    
        for k in tqdm_notebook(range(nb_sources + 1)):
            
            stop = (k == nb_sources)
            
            url = sources[k]
            
            journal = newspaper.build(url, memoize_articles=False)
            q = len(journal.articles)
        
            if q != 0:
            
                m = min(art_per_source,q)
            
                for i in tqdm_notebook(range(m)):
                
                    article = journal.articles[i]
                
                    try:
                        article.download()
                        article.parse()
                    
                        if article.publish_date and date1 <= article.publish_date <= date2 :
                        
                            article.nlp()
                            keywords = article.keywords
                            title = article.title
                            word_list = title.split() + keywords
                    
                            if all(w in word_list for w in keyword_list):
                                d += [[url, article.title, article.text]]
                                t += 1
                                
                                stop = (t == n)
                        
                                print((i+1, m, url, "found"))      #test
                        
                            else :
                        
                                print((i+1, m, url, "not found"))  #test
                            
                            stop = (k+1,i+1 == nb_sources, m)
                        
                    except:
                        
                        print((i+1, m, url, "error"))          #test
                            
                        stop = (k+1,i+1 == nb_sources, m)
                            
                        continue
                    
                    
    df = pd.DataFrame(d, columns=['Source', 'Title', 'Article'])
    #df.to_csv("scraping_by_keyword.csv", index = False)
    display(df)

In [ ]:
kwds = ["coronavirus","pandemic"]
n = 1
source_list = sourcetest2
min_date=[2020,1,1]

kw_scraping2(kwds, n, source_list, min_date)

In [ ]:
def kw_scraping3(keyword_list, n=1, source_list=[], min_date=[1,1,1], max_date=[9999,12,31], art_per_source= float("inf")):
    
    d=[]
    sources = source_list
   
    if not sources:
            sources = newspaper.popular_urls()
    
    date1 = datetime.datetime(min_date[0],min_date[1],min_date[2])
    date2 = datetime.datetime(max_date[0],max_date[1],max_date[2])
    
    nb_sources = len(sources)
    
    
    papers = []
    for url in tqdm_notebook(sources):
        papers.append(newspaper.build(url))
    news_pool.set(papers, threads_per_source=2)
    news_pool.join()
    
    t = 0
    
    stop = False
    
    while not stop :
    
        for k in tqdm_notebook(range(nb_sources + 1)):
            
            stop = (k == nb_sources)
            
            journal = newspaper.build(sources[k], memoize_articles=False)
            q = len(journal.articles)
        
            if q != 0:
            
                m = min(art_per_source,q)
            
                for i in range(m):
                
                    article = journal.articles[i]
                
                    try:
                        article.download()
                        article.parse()
                    
                        if not article.publish_date:
                            if date1 <= article.publish_date <= date2 :
                                
                                article.nlp()
                                keywords = article.keywords
                                title = article.title
                                word_list = title.split() + keywords
                                
                                if all(w in word_list for w in keyword_list):
                                    
                                    d += [[url, article.title, article.text]]
                                    t += 1
                                
                                    stop = (t == n)
                        
                                    print((i+1, m, url, "found"))      #test
                        
                                else :
                        
                                    print((i+1, m, url, "not found"))  #test
                            
                                stop = (k+1,i+1 == nb_sources, m)
                        
                    except:
                        
                        print((i+1, m, url, "error"))          #test
                            
                        stop = (k+1,i+1 == nb_sources, m)
                            
                        continue
                    
                    
    df = pd.DataFrame(d, columns=['Source', 'Title', 'Article'])
    #df.to_csv("scraping_by_keyword.csv", index = False)
    display(df)

In [3]:
def creation_base(n=500, source_list=[], i = 1000, nb_sujets = 500):
    
    d=[]
    sources = source_list
   
    if not sources:
        
        sources = newspaper.popular_urls()
        
    photos_raw = pd.read_csv("photos.csv",encoding='latin-1')
    photos = photos_raw.dropna(subset = ['keyword'])
    
    random_idx = np.random.choice(list(photos.index), nb_sujets, replace = False)
    random_kw = []
    for i in random_idx :
        random_kw += photos['keyword'][i]
    
    
    t = 0
        
    for url in tqdm_notebook(sources) :
        
        journal = newspaper.build(url, memoize_articles=False)
        q = len(journal.articles)
        
        if q != 0:
                        
            m = min(i,q)
            
            for i in tqdm_notebook(range(m)):
                                
                article = journal.articles[i]
                
                try:
                    
                    article.download()
                    article.parse()
                    article.nlp()
                    keywords = article.keywords
                    title = article.title
                    word_list = title.split() + keywords
                    
                    if any(w in word_list for w in random_kw):
                        d += [[url, title, article.text]]
                        t += 1
                        print(t)
                        if t == n:
                            break
                    
                except:
                    continue
        
        if t == n:
            break
            
            
    df = pd.DataFrame(d, columns=['Source', 'Title', 'Article'])
    df.to_csv("base_articles.csv", index = False)
    display(df)

In [4]:
creation_base()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (10,15,17,23,24,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233



234
235



236
237



,Source,Title,Article
0,http://cnn.com,Travel News - CNN,Outlook for Disney World and Disneyland reopenings: What will it be like? And when will parks be...
1,http://cnn.com,"80 million Chinese may already be out of work. 9 million more will soon be competing for jobs, too",The 26-year old tech worker spent the last year jumping from one startup to another in what alre...
2,http://cnn.com,Priyanka Chopra's bold fashion has made her a global style icon,"Written by Marianna Cerini, CNN\n\nPriyanka Chopra is an actress, producer, philanthropist, sing..."
3,http://cnn.com,Three Marines restrained an unruly passenger on a flight from Japan,(CNN) Three US Marines stepped in to detain a passenger who allegedly made threatening comments ...
4,http://cnn.com,A 'travel bubble' between New Zealand and Australia could be a model for the future,"Wellington, New Zealand (CNN) — It may be some time before tourists are traveling the globe agai..."
...,...,...,...
232,http://cnn.com,Fly in a WWI fighter plane,"Red Hook, New York (CNN) A century after the Armistice ended World War I, pilots take to the sky..."
233,http://www.time.com,How to Make a DIY Face Mask for Coronavirus,"Amid a shortage of medical-grade face masks, such as surgical masks (loose-fitting, disposable m..."
234,http://www.time.com,How to Make a DIY Face Mask for Coronavirus,"Amid a shortage of medical-grade face masks, such as surgical masks (loose-fitting, disposable m..."
235,http://www.ted.com,Esther Perel: Rethinking infidelity ... a talk for anyone who has ever loved,"You have JavaScript disabled\n\nFor the best experience, please turn JavaScript on. Here's how"


In [ ]:
photos_raw = pd.read_csv("photos.csv",encoding='latin-1')
photos = photos_raw.dropna(subset = ['keyword'])
list(photos.index)

In [ ]:
def creation_base_naive(n=100, source_list=[], keywordlist = [], i = 10000, nb_sujets = 100):
    d=[]
    sources = source_list
   
    if not sources:
        sources = newspaper.popular_urls()
    
    if not keywordlist:
        
        kwlist = newspaper.hot()      
        
        sources = newspaper.popular_urls()

    
    t = 0
    
    while t<n :
        
        for url in tqdm_notebook(sources) :
        
            journal = newspaper.build(url, memoize_articles=False)
            q = len(journal.articles)
        
            if q != 0:
                        
                m = min(i,q)
            
                for i in range(m):
                                
                    article = journal.articles[i]
                
                    try:
                    
                        article.download()
                        article.parse()
                        article.nlp()
                        keywords = article.keywords
                        title = article.title
                        word_list = title.split() + keywords
                        
                        if any(w in word_list for w in kwlist):
                   
                            d += [[url, title, article.text]]
                            
                            print(t)
                            
                            t += 1
                        
                            #print((i, m, url, "found"))      #test
                            #print(article.publish_date)      #test
                        
                        
                        #else :
                    
                            #print((i, m, url, "not found"))  #test
                            #print(article.publish_date)
                    
                    except:
                        test = 0
                        print((i, m, url, "error"))          #test
                        continue
                
    df = pd.DataFrame(d, columns=['Source', 'Title', 'Article'])
    df.to_csv("base_articles_naive.csv", index = False)
    display(df)

In [ ]:
creation_base_naive()